In [ ]:
from fenics import *
import matplotlib.pyplot as plt
from mshr import *
import numpy as np
import os

In [ ]:
os.chdir('/home/erick/Desktop/FEniCS/FEniCS-projects/diffution systems/predator-prey/fields')

In [ ]:
def split_fields(field, coordinates_size, mesh_triangulation):
    p0 = Point(0, 0)
    p1 = Point(coordinates_size,coordinates_size) 
    square = Rectangle(p0, p1)  
    mesh = RectangleMesh(p0, p1, mesh_triangulation, mesh_triangulation, "right/left")
    U = FiniteElement("CG", mesh.ufl_cell(), 2)
    V = FunctionSpace(mesh, U * U)
    v = Function(V)
    v.vector().set_local(fields)
    N, P = v.split()
    return N, P

In [ ]:
def fenics_field_to_numpy_array(fenics_field, coordinates_size): 
    field_array = np.empty((0, coordinates_size), int)
    lista = []
    for val_x in range(0,coordinates_size):
        for val_y in range(0,coordinates_size):
            valor=fenics_field(val_x,val_y)
            lista.append(valor)

            if len(lista) == coordinates_size:
                field_array = np.append(field_array, np.array([lista]), axis=0)
                if val_x < coordinates_size:
                    lista = []
    return field_array

In [ ]:
def power_spectrum(field):
    fft_field = np.fft.fft2(field)
    fft_field=np.fft.fftshift(fft_field)
    fft_field=(np.abs(fft_field))
    max_fft = np.where(fft_field == fft_field.max())
    fft_field[max_fft[0][0],max_fft[1][0]] = 0      
    return fft_field

In [ ]:
def plot_field_and_fft(field, field_type, fft_field, t):
    
    plt.figure(figsize=(11, 4))
    plt.subplot(1, 2, 1)
    if field_type == 'P':
        plt.imshow(field_array, cmap = 'seismic' , interpolation = 'bilinear')
        plt.title("$P(t={})$".format(t))
    if field_type == 'N':
        plt.imshow(field_array, cmap = 'gray' , interpolation = 'bilinear')
        plt.title("$N(t={})$".format(t))
    plt.colorbar() 
    
    plt.subplot(1, 2, 2)
    plt.imshow(fft_field, cmap = 'seismic' , interpolation = 'bilinear')
    if field_type == 'P':
            plt.title("$P_{fft}(t=%s)$" % t)
    if field_type == 'N':
            plt.title("$N_{fft}(t=%s)$" % t)
    plt.colorbar()
    plt.show()
    
    plt.tight_layout(pad=-1.0)
    plt.show()

In [ ]:
coordinates_size = 70
# Time parameters
dt = 5 # step
T = 500 # end step
t = 0  # zero time

In [ ]:
while t < T:
    t += dt
    fields=np.load('fields_{}.npy'.format(t))
    N,P = split_fields(fields, coordinates_size, 100)
    
    field_array = fenics_field_to_numpy_array(N, coordinates_size)
    fft_field = power_spectrum(field_array)
    plot_field_and_fft(field_array, 'N', fft_field, t)
    
    field_array = fenics_field_to_numpy_array(P, coordinates_size)
    fft_field = power_spectrum(field_array)
    plot_field_and_fft(field_array, 'P', fft_field, t)
